# **Documentation: Switching to PostgreSQL and Creating a Dockerized Django App**

This guide will walk you through the process of migrating your Django application from the default SQLite database to PostgreSQL and setting up Docker to containerize the application. 

---

## **1. Switch to PostgreSQL Database**

### **Step 1: Install PostgreSQL and Psycopg2**
PostgreSQL needs to be installed, and `psycopg2` is the PostgreSQL adapter for Python.

**Install psycopg2**:
```bash
pip install psycopg2
```

---

### **Step 2: Modify the Django Settings**
In the `settings.py` file of your Django project, change the `DATABASES` setting to use PostgreSQL instead of SQLite.

**Before (using SQLite):**
```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}
```

**After (using PostgreSQL):**
```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'your_db_name',
        'USER': 'your_db_user',
        'PASSWORD': 'your_db_password',
        'HOST': 'localhost',
        'PORT': '5432',
    }
}
```

- Replace `'your_db_name'`, `'your_db_user'`, and `'your_db_password'` with your PostgreSQL credentials.

---

### **Step 3: Create a PostgreSQL Database**

Before running migrations, you need to create a database and user in PostgreSQL.

- Log into PostgreSQL:
```bash
psql -U postgres
```

- Create a new database:
```sql
CREATE DATABASE your_db_name;
```

- Create a new user and set the password:
```sql
CREATE USER your_db_user WITH PASSWORD 'your_db_password';
```

- Grant privileges to the user on the database:
```sql
GRANT ALL PRIVILEGES ON DATABASE your_db_name TO your_db_user;
```

---

### **Step 4: Run Migrations**

Now, you need to apply migrations to set up your PostgreSQL database.

```bash
python manage.py migrate
```

---

## **2. Dockerize Your Django Application**

### **Step 1: Create a Dockerfile**

In the root directory of your Django project, create a `Dockerfile` to containerize the app. The `Dockerfile` should look like this:

```dockerfile
# Step 1: Use an official Python runtime as a parent image
FROM python:3.9

# Step 2: Set the working directory in the container
WORKDIR /app

# Step 3: Copy the current directory contents into the container at /app
COPY . /app/

# Step 4: Install any needed packages specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Step 5: Make port 8000 available to the world outside the container
EXPOSE 8000

# Step 6: Define environment variable
ENV PYTHONUNBUFFERED 1

# Step 7: Run the Django development server when the container launches
CMD ["python", "manage.py", "runserver", "0.0.0.0:8000"]
```

---

### **Step 2: Create a `docker-compose.yml` File**

To manage the Docker containers for Django and PostgreSQL, create a `docker-compose.yml` file.

```yaml
version: '3'

services:
  db:
    image: postgres:15
    environment:
      POSTGRES_DB: your_db_name
      POSTGRES_USER: your_db_user
      POSTGRES_PASSWORD: your_db_password
    ports:
      - "5432:5432"
    volumes:
      - postgres_data:/var/lib/postgresql/data

  web:
    build: .
    command: sh -c "python manage.py migrate && python manage.py runserver 0.0.0.0:8000"
    volumes:
      - .:/app
    ports:
      - "8000:8000"
    depends_on:
      - db

volumes:
  postgres_data:
```

- **`db`**: This service uses the official PostgreSQL image.
- **`web`**: This service builds the Django application using the `Dockerfile` and runs it.
- The `depends_on` ensures that PostgreSQL starts before Django.

---

### **Step 3: Build the Docker Containers**

Build the containers using the following command:

```bash
docker-compose up --build
```

This command will:

1. Build the Django application container from the Dockerfile.
2. Create the PostgreSQL container.
3. Run both containers.

---

### **Step 4: Access the Application**

Once the containers are built, access your Django app at `http://localhost:8000` in your browser.

---

### **Step 5: Verify PostgreSQL Connection**

Make sure your Django app is connected to PostgreSQL and the migration is successful by checking the logs of the Django container:

```bash
docker-compose logs web
```

You should see the server running, and you can check for any errors related to database connections.

---

### **Step 6: Stopping the Containers**

To stop the containers, you can use:

```bash
docker-compose down
```

---

## **3. Clean Up Docker Images**

After using Docker, you may want to clean up unused Docker images to free up space. To remove all unused images:

```bash
docker image prune -a
```

---

## **4. Conclusion**

In this documentation, we've successfully transitioned our Django app to use PostgreSQL and containerized the application with Docker. This will allow for easier deployment and management of your application. You can continue developing your app, and if you wish to deploy it, you can extend this Docker setup to use production-level configurations.

